# Estimate Mobile Phone Price
In this computer assignment, we try to building a model to estimate phone price using the given dataset.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive
# %mkdir AI_CA5
%cd AI-Final Project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
/content/drive/My Drive/AI-Final Project


#### Libraries
1. `pd` is imported to read dataset.

2. `LinearRegression`, `HuberRegressor` and `RandomForestRegressor` are imported to build models for our regression problem.

3. `mean_squared_error`, `mean_absolute_error` are imported to calculate the performance metrics.


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import KFold 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Data
There is a `mobile_phone_dataset.csv` file including a list of mobile advertisements with their information such as *title*, *brand*, *city*, *price* and etc.


In [ ]:
data = pd.read_csv("mobile_phone_dataset.csv")
data.head()

,Unnamed: 0,brand,city,title,desc,image_count,created_at,price
0,0,Nokia::نوکیا,Qom,نوکیا6303,سلام.یه گوشیه6303سالم که فقط دوتا خط کوچیک رو ...,2,Wednesday 07AM,60000
1,1,Apple::اپل,Tehran,ایفون ٥اس٣٢گیگ,درحد نو سالم اصلى بدون ضربه مهلت تست میدم,0,Wednesday 11AM,1150000
2,2,Samsung::سامسونگ,Mashhad,سامسونگ j5,گوشى بسیار بسیار تمیز و فقط سه هفته کارکرده و ...,2,Wednesday 02PM,590000
3,3,Apple::اپل,Karaj,گرى 5s ایفون 32گیگ,گلس پشت و رو .کارت اپل ای دی. لوازم جانبی اصلی...,3,Wednesday 04PM,1100000
4,4,Samsung::سامسونگ,Tehran,galaxy S5 Gold در حد آک,کاملا تمیز و بدون حتی 1 خط و خش\nبه همراه گلاس...,2,Friday 01PM,900000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59189 entries, 0 to 59188
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   59189 non-null  int64 
 1   brand        59189 non-null  object
 2   city         59189 non-null  object
 3   title        59189 non-null  object
 4   desc         59189 non-null  object
 5   image_count  59189 non-null  int64 
 6   created_at   59189 non-null  object
 7   price        59189 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 3.6+ MB


The following table shows some information of each class including the number of advertisements of each class and also the average price of each brand. 

In [ ]:
data.groupby("brand").describe()

Unnamed: 0                ...      price           
                      count          mean  ...        75%        max
brand                                      ...                      
Apple::اپل          15589.0  31093.334916  ...  1600000.0  2800000.0
HTC::اچ‌تی‌سی        2973.0  30982.716448  ...   600000.0  2000000.0
Huawei::هوآوی        7227.0  31160.014390  ...   410000.0  2500000.0
LG::ال‌جی            2490.0  31167.793574  ...   750000.0  2100000.0
Lenovo::لنوو         1100.0  31374.930909  ...   410000.0  2000000.0
Nokia::نوکیا         4027.0  30280.769556  ...   170000.0  2400000.0
Samsung::سامسونگ    19760.0  31161.525557  ...   850000.0  2700000.0
Sony::سونی           4227.0  31183.898746  ...   600000.0  2400000.0
ZTE::زدتی‌ای         1796.0  30885.280067  ...   500000.0  2400000.0

[9 rows x 24 columns]

## Preprocessing
In this part, `preprocessing()` is implemented to do the following tasks:
  1. Remove irrelevant coloumn
  2. Convert farsi and arabic numbers to english numeric characters.
  3. Remove brand names from `title`
  4. Devide the `title` of each advertisement into `persian_title` and `english title`
  5. Devide data into test and train
  6. Calculate **z_score** which shows how far is a price from average. Using this parameter, we can detect and remove outliers by selecting some thresholds. These thresholds are hyperparameters which is set manually. The following equation shows how z_score filters outliers.
$$ Z\_score = \frac{x-\eta}{\sigma}$$


In [ ]:
def preprocessing(data, redundant_words = ["iphone", "galaxy", "phone"], Upper_z_score = 2.5, Lower_z_score = -1):

  modified_data = data.filter(items=["brand", "title", "price"])

  trans_dict = dict()
  for i in range(10):
    trans_dict[i + ord('٠') ] = chr( i + ord('0'))
    trans_dict[i + ord('۰') ] = chr( i + ord('0'))

  modified_data["title"]= modified_data["title"].str.lower()
  modified_data["title"]= modified_data["title"].str.translate(trans_dict)


  brand_names = list(modified_data.groupby("brand").groups.keys())
  brand_names = ('::'.join(map(str, brand_names)).lower() + "::" + ('::'.join(map(str, redundant_words)).lower())).split("::")

  for w in brand_names:
    modified_data["title"] = modified_data["title"].str.replace(w, ' ') 

  modified_data.insert(1, "persian_title", modified_data["title"].str.replace(r'[\x00-\x7F]+'.lower(), ' '))
  modified_data.insert(2, "english_title", modified_data["title"].str.replace(r'[^A-Za-z0-9_]+'.lower(), ' '))
  modified_data = modified_data.drop(columns=["title"])

  test_data = modified_data[modified_data['price'] == -1]
  train_data = modified_data[modified_data['price'] != -1]

  if Upper_z_score and Lower_z_score:
    train_data_dict = dict()

    for name, group in train_data.groupby('brand'):
      group.insert(3, "z_score", stats.zscore(group['price']).tolist())
      train_data_dict[name] = group

      train_data_dict[name] = train_data_dict[name][train_data_dict[name]["z_score"] > -1]
      train_data_dict[name] = train_data_dict[name][train_data_dict[name]["z_score"] < 2.5]

  return train_data_dict, test_data

In the following function, the most `num` frequent words in the given `col` for all brands will be found and returned.

In [ ]:
def find_key_words(train_data_dict, col = 'english_title', num = 40):
  key_words = dict()
  freq_key = dict()
  for key, val in train_data_dict.items():
    word_list = ' '.join(map(str, val[col].tolist() )).split()
    word_freq = dict()
    for w in set(word_list):
      word_freq[w] = word_list.count(w)
    freq, word = zip(*sorted(zip(word_freq.values(), word_freq.keys()), reverse=True))
    key_words[key] = list(word[:num])
    freq_key[key] = list(freq[:num])

  return key_words, freq_key

`extract_features` add some zero coloumns according to the given `key_words`. If `persian_title` or `english_title` contain such key word, the corresponding value gets 1.

In [ ]:
def extract_features(train_data_dict, key_words, col = "english_title"):
  for key, val in train_data_dict.items():
    for w in key_words[key]:
      val[w] = val[col].str.contains(w).astype(int)
  return train_data_dict

## Evaluating Model
Features extracted in the previous section are used to predict the price of validation data. 

Using k-fold cross validation method, the average score of such trained model,  MSE and MAE are calculated by `evaluate_model`.


In [ ]:
def evaluate_model(train_data_dict, key_words, regression_method = RandomForestRegressor, K = 5):
  
  MSE = {b:0 for b in train_data_dict}
  MAE = {b:0 for b in train_data_dict}
  scores = {b:0 for b in train_data_dict}
  
  for brand in train_data_dict:
    X = train_data_dict[brand][key_words[brand]].to_numpy()
    y = train_data_dict[brand]["price"].to_numpy()

    kf = KFold(n_splits=K)
    for train_index, test_index in kf.split(X):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
      reg = regression_method().fit(X_train, y_train)

      scores[brand] += reg.score(X_test, y_test)
      MSE[brand] += mean_squared_error(y_test, reg.predict(X_test))
      MAE[brand] +=  mean_absolute_error(y_test, reg.predict(X_test))

    scores[brand] /= K
    MSE[brand] /= K
    MAE[brand] /= K
    
  return pd.DataFrame.from_dict({"score" : scores, "MSE" : MSE, "MAE" : MAE} , orient='index').T


`train_models()` returns the best models trained by all data.

In [ ]:
def train_models(train_data_dict, key_words, regression_method = RandomForestRegressor):
  models = dict()

  for brand in train_data_dict:
    X = train_data_dict[brand][key_words[brand]].to_numpy()
    y = train_data_dict[brand]["price"].to_numpy()

    models[brand] = regression_method().fit(X, y)

  return models

`predict()` estimate prices for given `test_data` using pretrained models.

In [ ]:
def predict(test_data, key_words, models):
  predicted_prices = pd.DataFrame(0, index=test_data.index, columns=["price"])
  for index, row in test_data.iterrows():
    brand_name = row["brand"]
    # print(brand_name, "**", row["english_title"])
    X = np.array([[1 if key_words[brand_name][i] in row["english_title"] else 0 for i in range(len(key_words[brand_name])) ]])
    # print(X)
    # print(models[brand_name].predict(X))
    predicted_prices.at[index, "price"] = models[brand_name].predict(X)

  return predicted_prices

## Results
In the following part, results are shown and explained.

In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, freq = find_key_words(train_data_dict, col = "persian_title")
print(key_words.values())
print(freq.values())

dict_values([['ایفون', 'گیگ', 'آیفون', 'گوشی', 'اس', 'گلد', 'موبایل', 'در', 'حد', 'پلاس', 'تمیز', 'گری', 'نو', 'فروش', 'مشکی', 'فایو', 'سیکس', 'و', 'سالم', 'گوشى', 'معاوضه', 'سیلور', 'سفید', 'بدون', 'اک', 'آک', 'با', 'درحد', 'بسیار', 'رنگ', '،', 'رزگلد', 'خط', 'خش', 'اکبند', 'آکبند', 'حافظه', 'اصلی', 'فوق', 'العاده'], ['گوشی', 'اچ', 'تی', 'سی', 'دیزایر', 'موبایل', 'وان', 'فروش', 'تمیز', 'دوسیم', 'نو', 'در', 'حد', 'سیم', 'و', 'ام', 'مدل', 'معاوضه', 'دو', 'سالم', 'پلاس', 'فورجی', 'آک', 'با', 'ای', 'درحد', 'یا', 'گیگ', 'تعویض', 'کارت', 'اک', 'جی', 'ایکس', 'بسیار', 'کارکرد', 'اس', 'مکس', 'فوق', 'تایوان', 'آکبند'], ['هواوی', 'گوشی', 'موبایل', 'جی', 'تمیز', 'هانر', 'مدل', 'در', 'حد', 'لایت', 'سالم', 'نو', 'فروش', 'وای', 'پی', 'معاوضه', 'و', 'هونور', 'سفید', 'اک', 'دوسیم', 'درحد', 'سی', 'مشکی', 'هو', 'آک', 'با', 'میت', 'هاوایی', 'هوواوی', 'یا', 'سیم', 'گلد', 'بسیار', 'دو', 'هواویی', 'هواووی', 'فورجی', 'وی', 'بدون'], ['جی', 'گوشی', 'ال', 'الجی', 'موبایل', 'گیگ', 'فروش', 'تمیز', 'معاوضه', 'مدل'

According to the above cell, the most frequent word in Persian doesn't carry significant semantic information. Therefore, in the rest of the project, English keywords are used to accomplish our task.

In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, freq = find_key_words(train_data_dict, col = "english_title")
print(key_words.values())
print(freq.values())

dict_values([['6', '5s', '6s', '64', '16', '5', 'gold', '32', '16g', 'gray', 'lla', '64g', '64gb', '16gb', 'plus', 'se', 'gig', '32g', '128', 'iphon', '64gig', '16gig', 'silver', 'num', 'gb', '5c', '32gb', 'g', 's', '32gig', '4s', '6plus', 'rose', 'grey', 'numg', '128g', 's5', 'i', 'black', 'a'], ['one', 'desire', '816', 'm8', 'dual', 'e9', 'sim', '820', 'm9', '4g', 'plus', 'm7', '826', '626', '616', 'x', 'eye', 'max', '500', 'e8', 'a9', '9', '32', '820s', 'gold', 'xl', 's', '728', 'g', '8', 'sensation', '820g', '310', 'disire', '8x', '700', '2', '4', '32g', 'xe'], ['g610', 'honor', 'p8', '4c', '3c', '6', 'g730', 'lite', 'g630', 'g6', 'p7', 'g750', '8', 'y600', '7', '610', 'p6', 'g700', 'y300', '730', 'y530', 'y6', 'y330', 'y520', '4x', 'y511', 'y5', 'g510', '4g', '750', 'g525', '4', 'y625', '5x', 'y560', 'u10', 'g', '630', 'g8', '600'], ['g3', 'g4', '3', '2', '4', 'g2', '32', 'k10', 'v10', '5', '4g', 'nexus', 'g5', 'stylus', 'g', '5x', '10', '32gig', '32g', 'beat', 'k4', 'x', 'screen'

The following cell shows the dataframe of `Apple::اپل` brand after finding and storing key words in OneHotEncoding method.

In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, _ = find_key_words(train_data_dict)
train_data_dict = extract_features(train_data_dict, key_words)
train_data_dict["Apple::اپل"]

,brand,persian_title,english_title,z_score,price,6,5s,6s,64,16,5,gold,32,16g,gray,lla,64g,64gb,16gb,plus,se,gig,32g,128,iphon,64gig,16gig,silver,num,gb,5c,32gb,g,s,32gig,4s,6plus,rose,grey,numg,128g,s5,i,black,a
1,Apple::اپل,ایفون اس گیگ,5 32,-0.041854,1150000,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Apple::اپل,گرى ایفون گیگ,5s 32,-0.125737,1100000,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,Apple::اپل,ایفون,6 64,0.880861,1700000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Apple::اپل,ايفون,5s,-0.545153,850000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,Apple::اپل,ایفون گیگ,se16,0.545328,1500000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59151,Apple::اپل,گیگ,6s 16,0.964744,1750000,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
59157,Apple::اپل,ایفون پلاس گیگ گلد,6 16,0.796977,1650000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59175,Apple::اپل,,iphon 6plus 128silver,1.468043,2050000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
59179,Apple::اپل,ایفون اس گیگ,6 64,1.300276,1950000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Regression Model
In the following section, `LinearRegression`, `HuberRegressor` and `RandomForestRegressor` are used to solve our regression problem.

* `LinearRegression`

In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, _ = find_key_words(train_data_dict)
train_data_dict = extract_features(train_data_dict, key_words)
evaluate_model(train_data_dict, key_words, regression_method = LinearRegression)

,score,MSE,MAE
Apple::اپل,0.753781,6.128895e+10,172898.963407
HTC::اچ‌تی‌سی,0.553760,2.435694e+10,113677.380423
Huawei::هوآوی,0.498759,1.298625e+10,77449.920145
LG::ال‌جی,0.447504,4.451712e+10,149011.006538
Lenovo::لنوو,0.521530,1.131945e+10,75235.265347
Nokia::نوکیا,0.627620,1.108800e+10,66462.493844
Samsung::سامسونگ,0.504658,7.703504e+10,198941.052440
Sony::سونی,0.514271,2.338261e+10,105486.009669
ZTE::زدتی‌ای,0.330069,6.993699e+10,198983.104264


* `HuberRegressor`: The Huber Regressor optimizes the squared loss for the samples where |(y - X'w) / sigma| < epsilon and the absolute loss for the samples where |(y - X'w) / sigma| > epsilon, where w and sigma are parameters to be optimized. The parameter sigma makes sure that if y is scaled up or down by a certain factor, one does not need to rescale epsilon to achieve the same robustness.




In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, _ = find_key_words(train_data_dict)
train_data_dict = extract_features(train_data_dict, key_words)
evaluate_model(train_data_dict, key_words, regression_method = HuberRegressor)

,score,MSE,MAE
Apple::اپل,0.750959,6.198703e+10,171321.914842
HTC::اچ‌تی‌سی,0.526137,2.586829e+10,113082.990921
Huawei::هوآوی,0.468082,1.379326e+10,75385.713813
LG::ال‌جی,0.417426,4.721033e+10,154870.414490
Lenovo::لنوو,0.448522,1.319556e+10,79185.928388
Nokia::نوکیا,0.523467,1.429369e+10,68560.446108
Samsung::سامسونگ,0.470431,8.234810e+10,189717.445534
Sony::سونی,0.490215,2.453676e+10,105933.352219
ZTE::زدتی‌ای,0.234965,8.011723e+10,199172.774820


* `RandomForestRegressor` : A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. 


In [ ]:
train_data_dict, test_data = preprocessing(data)
key_words, _ = find_key_words(train_data_dict)
train_data_dict = extract_features(train_data_dict, key_words)
evaluate_model(train_data_dict, key_words, regression_method = RandomForestRegressor)

,score,MSE,MAE
Apple::اپل,0.766467,5.813713e+10,164471.237611
HTC::اچ‌تی‌سی,0.571502,2.342893e+10,107732.538916
Huawei::هوآوی,0.552939,1.158724e+10,71550.016464
LG::ال‌جی,0.472811,4.251581e+10,139537.315853
Lenovo::لنوو,0.568291,1.027374e+10,70048.341168
Nokia::نوکیا,0.640423,1.071111e+10,64330.880977
Samsung::سامسونگ,0.583941,6.468059e+10,168304.764903
Sony::سونی,0.543501,2.191183e+10,99326.183548
ZTE::زدتی‌ای,0.329129,7.006821e+10,189190.438687


Comparing the trained models, `RandomForestRegressor` has the best performance.

### Test Prediction Result
Selecting best regression method, the following table shows the estimated prices for those with "-1" price value.

In [ ]:
models = train_models(train_data_dict, key_words)
predict(test_data, key_words, models)

,price
23,469416
28,997883
32,1185780
36,384402
44,279414
...,...
59121,241852
59146,473747
59169,1676528
59174,727166


## Conclusions and future work
In this computer assignment, `desc` column data were not used because of the informality of sentences which may be useful. Using semantic analysis, this valuable information would be extracted. Moreover, using 'hazm' library and 'TF-IDF' would be beneficial.

